# **Dexcom Clarity Readings - SQLite with SQL Magic**

This Python program automates the addition of new glucose readings from a CSV file into an SQLite database while ensuring data consistency and integrity. It automatically detects the latest CSV file, eliminating the need to manually rename it or move it to the program directory. If a new file is found, the program loads and processes the data by identifying and removing duplicates, validating date continuity, and restricting stored readings to the most recent 90 days. Older entries are deleted to optimize storage. The program then exports the latest 90 days of readings to a CSV file for Tableau visualizations.

Run on Python 3.13 | No errors | No warnings

In [1]:
# Import packages

# For data manipulation
import numpy as np
import pandas as pd

# For working with datetime objects
from datetime import datetime

# For working with SQLite databases
import sqlite3

# For working with files
import glob
import os

# For working with system commands
import sys

In [2]:
# Load the SQL Magic extension
%load_ext sql

In [3]:
# Create connection to the SQLite database
%sql sqlite:///ClarityHistory.db

Connecting to 'sqlite:///ClarityHistory.db'

In [4]:
# Get the record count for CLARITY_DATA
%sql SELECT COUNT(*) FROM CLARITY_DATA

Running query in 'sqlite:///ClarityHistory.db'

COUNT(*)
25702


In [5]:
# Display the first 5 rows of the CLARITY_DATA table
%sql SELECT * FROM CLARITY_DATA LIMIT 5

Running query in 'sqlite:///ClarityHistory.db'

Date,Time,DateTime,Value,Treatment,Source
2024-12-29,12:04 AM,2024-12-29 00:04:16,93,Mounjaro 12.5,CGM
2024-12-29,12:09 AM,2024-12-29 00:09:16,91,Mounjaro 12.5,CGM
2024-12-29,12:14 AM,2024-12-29 00:14:17,90,Mounjaro 12.5,CGM
2024-12-29,12:19 AM,2024-12-29 00:19:17,89,Mounjaro 12.5,CGM
2024-12-29,12:24 AM,2024-12-29 00:24:16,84,Mounjaro 12.5,CGM


In [6]:
# Display column names and data types for the CLARITY_DATA table
%sql PRAGMA table_info(CLARITY_DATA)

Running query in 'sqlite:///ClarityHistory.db'

cid,name,type,notnull,dflt_value,pk
0,Date,TEXT,0,None,0
1,Time,TEXT,0,None,0
2,DateTime,TEXT,0,None,0
3,Value,INTEGER,0,None,0
4,Treatment,TEXT,0,None,0
5,Source,TEXT,0,None,0


In [7]:
%%sql
-- Display the first and last dates in the table
SELECT MIN(DATE(Date)) AS Min_Date, MAX(DATE(Date)) AS Max_Date
FROM CLARITY_DATA;

Running query in 'sqlite:///ClarityHistory.db'

Min_Date,Max_Date
2024-12-29,2025-03-28


In [8]:
# Get the number of unique days in the CLARITY_DATA table
%sql SELECT COUNT(DISTINCT Date) FROM CLARITY_DATA

Running query in 'sqlite:///ClarityHistory.db'

COUNT(DISTINCT Date)
90


In [9]:
# Find the latest Clarity data file

# Define the directory and filename pattern
directory = r"C:\Users\clock\Downloads"
pattern = "Clarity_Export_Smith_Jeffrey_*.csv"

# Construct the full search pattern
search_pattern = os.path.join(directory, pattern)

# Find all matching files
matching_files = glob.glob(search_pattern)

# Ensure there is at least one match
if not matching_files:
    raise SystemExit("No matching files found. Exiting program.")

# Sort files by modification time to get the most recent one
latest_file = max(matching_files, key=os.path.getmtime)

# Extract just the file name
file_name = os.path.basename(latest_file)

print(f"Found file: {file_name}")

Found file: Clarity_Export_Smith_Jeffrey_2025-03-30_124811.csv


In [10]:
# Load new Clarity data
df0 = pd.read_csv(latest_file, usecols=['Index', 'Timestamp (YYYY-MM-DDThh:mm:ss)', 'Event Type', 'Glucose Value (mg/dL)'])

In [11]:
# Display the first 5 rows of the dataframe (df0)
df0.head()

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Glucose Value (mg/dL)
0,1,NaN,FirstName,NaN
1,2,NaN,LastName,NaN
2,3,NaN,Device,NaN
3,4,2025-03-29T00:00:08,EGV,105.0
4,5,2025-03-29T00:05:08,EGV,109.0


In [12]:
# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Index                            286 non-null    int64  
 1   Timestamp (YYYY-MM-DDThh:mm:ss)  283 non-null    object 
 2   Event Type                       286 non-null    object 
 3   Glucose Value (mg/dL)            283 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.1+ KB


In [13]:
# Keep only the rows where Event Type is EGV (estimated glucose value)
df0 = df0[df0['Event Type'] == 'EGV']

In [14]:
# Rename Glucose Value (mg/dL) to Value
df0 = df0.rename(columns={'Glucose Value (mg/dL)': 'Value'})

In [15]:
# Replace Value with 40 where it is 'Low'
df0['Value'] = df0['Value'].replace('Low', 40)

# Replace Value with 400 where it is 'High'
df0['Value'] = df0['Value'].replace('High', 400)

In [16]:
# Convert Value to integer
df0['Value'] = df0['Value'].astype(int)

In [17]:
# Rename and format Timestamp column
df0['DateTime'] = pd.to_datetime(df0.pop('Timestamp (YYYY-MM-DDThh:mm:ss)'), format='%Y-%m-%dT%H:%M:%S')

In [18]:
# Delete the Index column
df0 = df0.drop(['Index'], axis=1)

In [19]:
# Create a Date column from the DateTime column
df0['Date'] = df0['DateTime'].dt.date

In [20]:
# Create a Time column from the DateTime column
df0['Time'] = df0['DateTime'].dt.time

In [ ]:
# Create a column for Treatment
df0['Date'] = pd.to_datetime(df0['Date']) # Convert Date from text to datetime format

#Set the conditions and choices
conditions = [
    (df0['Date'] > '2025-02-25'),
    (df0['Date'] > '2024-06-18'),
    (df0['Date'] > '2024-01-30'),
    (df0['Date'] > '2023-07-18'),
    (df0['Date'] > '2023-02-28'),
    (df0['Date'] > '2023-01-31')
]

choices = [
    'Mounjaro 15',
    'Mounjaro 12.5',
    'Mounjaro 10',
    'Mounjaro 7.5',
    'Mounjaro 5',
    'Mounjaro 2.5'
]

df0['Treatment'] = np.select(conditions, choices, default='Untreated')

In [22]:
# Format Time as AM/PM
df0['Time'] = df0['Time'].apply(lambda x: x.strftime('%I:%M %p'))

In [23]:
# Make Sure Date is only the date, not date and time
df0['Date'] = df0['Date'].dt.date

In [24]:
# Drop rows where Date is current date
today = datetime.today().date()
df0 = df0[df0['Date'] < today]

In [25]:
# Reorder the columns
df0 = df0[['Date', 'Time', 'DateTime', 'Value', 'Treatment']]

In [26]:
# Add a column for Source
df0['Source'] = 'CGM'

In [27]:
# Find the min and max dates in the dataset
min_date0 = df0['Date'].min()
max_date0 = df0['Date'].max()

# Count the number of unique days in the dataset
unique_days = df0['Date'].nunique()

# Display the results
print("New Data")
print("Min Date:", min_date0)
print("Max Date:", max_date0)
print(f"Days in Dataset: {unique_days}")

# Count the number of unique days in the dataset
unique_days = df0['Date'].nunique()

New Data
Min Date: 2025-03-29
Max Date: 2025-03-29
Days in Dataset: 1


In [28]:
# Display the first 5 rows of the dataframe (df0)
df0.head()

,Date,Time,DateTime,Value,Treatment,Source
3,2025-03-29,12:00 AM,2025-03-29 00:00:08,105,Mounjaro 15,CGM
4,2025-03-29,12:05 AM,2025-03-29 00:05:08,109,Mounjaro 15,CGM
5,2025-03-29,12:10 AM,2025-03-29 00:10:07,107,Mounjaro 15,CGM
6,2025-03-29,12:15 AM,2025-03-29 00:15:07,108,Mounjaro 15,CGM
7,2025-03-29,12:20 AM,2025-03-29 00:20:07,109,Mounjaro 15,CGM


In [29]:
# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283 entries, 3 to 285
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       283 non-null    object        
 1   Time       283 non-null    object        
 2   DateTime   283 non-null    datetime64[ns]
 3   Value      283 non-null    int64         
 4   Treatment  283 non-null    object        
 5   Source     283 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 15.5+ KB


In [30]:
# Display basic statistics about the data
df0.describe()

,DateTime,Value
count,283,283.000000
mean,2025-03-29 11:47:43.077738496,126.038869
min,2025-03-29 00:00:08,77.000000
25%,2025-03-29 05:52:38,109.000000
50%,2025-03-29 11:45:10,114.000000
75%,2025-03-29 17:37:41,134.500000
max,2025-03-29 23:55:12,240.000000
std,NaN,30.356661


In [31]:
# Reindex df0
df0 = df0.reset_index(drop=True)

In [32]:
# Add the new data to the existing CLARITY_DATA table
df0.to_sql('CLARITY_DATA', con=sqlite3.connect('ClarityHistory.db'), if_exists='append', index=False)

283

In [33]:
%%sql
-- Find the total count of duplicate rows in the CLARITY_DATA table
SELECT SUM(duplicate_count - 1) AS total_duplicates
FROM (
    SELECT COUNT(*) AS duplicate_count
    FROM CLARITY_DATA
    GROUP BY Date, Time, DateTime, Value, Treatment, Source
    HAVING COUNT(*) > 1
) as duplicates;

Running query in 'sqlite:///ClarityHistory.db'

total_duplicates
None


In [34]:
%%sql
-- Delete duplicate rows in the CLARITY_DATA table
DELETE FROM CLARITY_DATA
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM CLARITY_DATA
    GROUP BY Date, Time, DateTime, Value, Treatment, Source
);

Running query in 'sqlite:///ClarityHistory.db'

++
||
++
++

In [35]:
%%sql
-- Find missing dates in CLARITY_DATA
WITH DateRange AS (
    -- Generate a range of consecutive dates between the minimum and maximum dates in the dataset
    SELECT date(min(Date)) as StartDate, date(max(Date)) as EndDate
    FROM CLARITY_DATA
),
AllDates AS (
    -- Recursive query to generate all dates between StartDate and EndDate
    SELECT StartDate as Date
    FROM DateRange
    UNION ALL
    SELECT date(Date, '+1 day')
    FROM AllDates, DateRange
    WHERE Date < EndDate
)
-- Find dates in the generated date range that do not exist in your table
SELECT Date
FROM AllDates
WHERE Date NOT IN (SELECT DISTINCT Date FROM CLARITY_DATA);

Running query in 'sqlite:///ClarityHistory.db'

Date


In [36]:
%%sql
-- Delete rows in CLARITY_DATA that are older than 90 days
DELETE FROM CLARITY_DATA
WHERE Date < (SELECT date(MAX(Date), '-89 day') FROM CLARITY_DATA);

Running query in 'sqlite:///ClarityHistory.db'

286 rows affected.

++
||
++
++

In [37]:
# Retrieve all data from the CLARITY_DATA table
df = %sql SELECT * FROM CLARITY_DATA

# Convert the ResultSet to a DataFrame
df1 = df.DataFrame()

Running query in 'sqlite:///ClarityHistory.db'

In [38]:
# Display the first 5 rows of the dataframe (df1)
df1.head()

,Date,Time,DateTime,Value,Treatment,Source
0,2024-12-30,12:04 AM,2024-12-30 00:04:20,96,Mounjaro 12.5,CGM
1,2024-12-30,12:09 AM,2024-12-30 00:09:20,97,Mounjaro 12.5,CGM
2,2024-12-30,12:14 AM,2024-12-30 00:14:20,98,Mounjaro 12.5,CGM
3,2024-12-30,12:19 AM,2024-12-30 00:19:20,97,Mounjaro 12.5,CGM
4,2024-12-30,12:24 AM,2024-12-30 00:24:20,96,Mounjaro 12.5,CGM


In [39]:
# Display basic information about the data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25699 entries, 0 to 25698
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       25699 non-null  object
 1   Time       25699 non-null  object
 2   DateTime   25699 non-null  object
 3   Value      25699 non-null  int64 
 4   Treatment  25699 non-null  object
 5   Source     25699 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [40]:
# Get the min and max dates
min_date = df1['Date'].min()
max_date = df1['Date'].max()

print ("Combined Data")
print("Min Date:", min_date)
print("Max Date:", max_date)

# Create a complete date range from min to max date
complete_date_range = pd.date_range(start=min_date, end=max_date, freq='D')

# Get the unique dates from your dataset
unique_dates = pd.to_datetime(df1['Date']).sort_values().unique()

# Check for missing dates using numpy set difference
missing_dates = np.setdiff1d(complete_date_range, unique_dates)

# Display missing dates, if any
if len(missing_dates) > 0:
    print("Missing dates:")
    print(missing_dates)
else:
    print("No missing dates, all dates are consecutive.")

Combined Data
Min Date: 2024-12-30
Max Date: 2025-03-29
No missing dates, all dates are consecutive.


In [41]:
# Count the number of unique days in the dataset
unique_days1 = df1['Date'].nunique()

# Count the number of unique days in the dataset that have at least 144 readings
days_with_144_readings = df1['Date'].value_counts().ge(144).sum()

# Display the results
print(f"Days in Dataset: {unique_days1}")
print(f"Days With at Least 144 Readings: {days_with_144_readings}")

Days in Dataset: 90
Days With at Least 144 Readings: 90


In [42]:
# Get a count of readings per day
readings_per_day = df1['Date'].value_counts()

# Identify the days with fewer than 144 readings
days_below_144_readings = readings_per_day[readings_per_day < 144]

# Sort the days by date in ascending order
days_below_144_readings = days_below_144_readings.sort_index()

# Format the output
num_days_below_144 = len(days_below_144_readings)
formatted_output = days_below_144_readings.reset_index()

# Display the results
print(f"Days with Fewer Than 144 Readings: {num_days_below_144}")
for date, count in formatted_output.values:
    print(f"{date}: {count}")

Days with Fewer Than 144 Readings: 0


In [43]:
# Verify the changes
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25699 entries, 0 to 25698
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       25699 non-null  object
 1   Time       25699 non-null  object
 2   DateTime   25699 non-null  object
 3   Value      25699 non-null  int64 
 4   Treatment  25699 non-null  object
 5   Source     25699 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.2+ MB


In [44]:
# Save df1 to a csv file
df1.to_csv('Clarity Readings for Analysis.csv', index=False)